conda install -c conda-forge liac-arff
pip install scipy

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import arff

In [52]:
# load the ARFF file
with open('Data\Training_Dataset.arff', 'r') as f:
    data, meta = arff.loadarff(f)

# convert to list of lists
data_list = data.tolist()

# convert to pandas DataFrame
df = pd.DataFrame(data_list)

# add attribute names as columns
df.columns = meta.names()

# iterate over the DataFrame and convert each value to a string
for col in df.columns:
    df[col] = df[col].apply(lambda x: x.decode() if type(x) == bytes else x)

df = df.astype(float)

# save the DataFrame to a new csv file
# df.to_csv('Data/new_file.csv', index=False)

# split the dataframe into train and test sets
# train_df, test_df = train_test_split(df, test_size=0.25, random_state=2023)
X_train, X_test, y_train, y_test = train_test_split(df.drop('Result', axis=1), df['Result'], test_size=0.25)

# count how many records are there in DataFrame
num_rows = df.shape[0]

# count how many records are there in train DataFrame
num_rows_train = X_train.shape[0]

# count how many records are there in DataFrame
num_rows_test = X_test.shape[0]

print('Train df size is :' , num_rows_train)
print('Test df size is :' , num_rows_test)
print('Whole df size is :' ,num_rows)
print('Test df + Train df size is :' , num_rows_train + num_rows_test)

Train df size is : 8291
Test df size is : 2764
Whole df size is : 11055
Test df + Train df size is : 11055


In [47]:
# liczenie ilości próbek w każdej z klas
class_counts = df['Result'].value_counts()

# obliczenie proporcji próbek w każdej z klas
class_proportions = class_counts / class_counts.sum()

print(class_proportions)

 1.0    0.556943
-1.0    0.443057
Name: Result, dtype: float64


In [48]:
# get a list of all the column names
attributes_test = df.columns.tolist()

# print the list of column names
print(attributes_test)

['having_IP_Address', 'URL_Length', 'Shortining_Service', 'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix', 'having_Sub_Domain', 'SSLfinal_State', 'Domain_registeration_length', 'Favicon', 'port', 'HTTPS_token', 'Request_URL', 'URL_of_Anchor', 'Links_in_tags', 'SFH', 'Submitting_to_email', 'Abnormal_URL', 'Redirect', 'on_mouseover', 'RightClick', 'popUpWidnow', 'Iframe', 'age_of_domain', 'DNSRecord', 'web_traffic', 'Page_Rank', 'Google_Index', 'Links_pointing_to_page', 'Statistical_report', 'Result']


In [49]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8291 entries, 5586 to 6479
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   having_IP_Address            8291 non-null   float64
 1   URL_Length                   8291 non-null   float64
 2   Shortining_Service           8291 non-null   float64
 3   having_At_Symbol             8291 non-null   float64
 4   double_slash_redirecting     8291 non-null   float64
 5   Prefix_Suffix                8291 non-null   float64
 6   having_Sub_Domain            8291 non-null   float64
 7   SSLfinal_State               8291 non-null   float64
 8   Domain_registeration_length  8291 non-null   float64
 9   Favicon                      8291 non-null   float64
 10  port                         8291 non-null   float64
 11  HTTPS_token                  8291 non-null   float64
 12  Request_URL                  8291 non-null   float64
 13  URL_of_Anchor  

In [50]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2764 entries, 2497 to 3388
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   having_IP_Address            2764 non-null   float64
 1   URL_Length                   2764 non-null   float64
 2   Shortining_Service           2764 non-null   float64
 3   having_At_Symbol             2764 non-null   float64
 4   double_slash_redirecting     2764 non-null   float64
 5   Prefix_Suffix                2764 non-null   float64
 6   having_Sub_Domain            2764 non-null   float64
 7   SSLfinal_State               2764 non-null   float64
 8   Domain_registeration_length  2764 non-null   float64
 9   Favicon                      2764 non-null   float64
 10  port                         2764 non-null   float64
 11  HTTPS_token                  2764 non-null   float64
 12  Request_URL                  2764 non-null   float64
 13  URL_of_Anchor  

In [10]:
# # Convert the test data into a format that can be used by the perceptron
# X_test = test_df.values
#
# # Encode the categorical variables in the test data
# encoder = LabelEncoder()
# for i in range(X_train.shape[1]):
#     if type(X_train[0, i]) is str:
#         X_train[:, i] = encoder.fit_transform(X_train[:, i])
#         X_test[:, i] = encoder.transform(X_test[:, i])
#
# # Make predictions on the test data
# y_pred = perceptron.predict(X_test)
#
# correct_predictions = (y_pred == result_test_df).sum()
#
#
# print(correct_predictions , 'out of ',num_rows_test)
# print(correct_predictions / num_rows_test)

1453 out of  2764
0.5256874095513748


In [51]:
from keras.models import Sequential
from keras.layers import Dense

# Tworzenie sieci
model = Sequential()
model.add(Dense(32, input_dim=30, activation='relu')) # warstwa wejściowa, 32 neuronów, 31 wejść
model.add(Dense(16, activation='relu')) # pierwsza warstwa ukryta, 16 neuronów
model.add(Dense(8, activation='relu')) # druga warstwa ukryta, 8 neuronów
model.add(Dense(1, activation='sigmoid')) # warstwa wyjściowa, 1 neuron, funkcja aktywacji sigmoid

# Kompilacja sieci
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train,y_train, epochs =20, batch_size=32, validation_data=(X_test,y_test))
test_loss, test_acc = model.evaluate(X_test,y_test)
print('Test accuracy: ', test_acc)

Epoch 1/20
260/260 [==============================] - 2s 3ms/step - loss: -14.0977 - accuracy: 0.1907 - val_loss: -73.3645 - val_accuracy: 0.3173
Epoch 2/20
260/260 [==============================] - 0s 2ms/step - loss: -658.2969 - accuracy: 0.2900 - val_loss: -1789.1355 - val_accuracy: 0.3119
Epoch 3/20
260/260 [==============================] - 1s 2ms/step - loss: -5218.9609 - accuracy: 0.2924 - val_loss: -9770.4238 - val_accuracy: 0.3079
Epoch 4/20
260/260 [==============================] - 1s 2ms/step - loss: -19218.4512 - accuracy: 0.2892 - val_loss: -29716.7031 - val_accuracy: 0.3086
Epoch 5/20
260/260 [==============================] - 0s 2ms/step - loss: -49796.7891 - accuracy: 0.2872 - val_loss: -69323.0469 - val_accuracy: 0.3285
Epoch 6/20
260/260 [==============================] - 0s 2ms/step - loss: -103978.0859 - accuracy: 0.2930 - val_loss: -134497.0625 - val_accuracy: 0.3086
Epoch 7/20
260/260 [==============================] - 1s 2ms/step - loss: -189109.1875 - accuracy